# Extract audio features from each song

In [11]:
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os
from time import sleep  

In [12]:
SPOTIPY_CLIENT_ID='e1a52feaafdd43ad9eff6dccd21a966e'
SPOTIPY_CLIENT_SECRET='3581d4abb2d9434489ed89cd3160b90e'

client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace = False

In [13]:
ranking_features_df= pd.read_csv(os.path.join('raw_data', 'ranking_all_2017_2022.csv'), index_col=0, parse_dates=['Date', 'release_date'])
ranking_features_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,song_id,release_date
0,1,Échame La Culpa,"Luis Fonsi, Demi Lovato",26459,1zsG4eaZmkA1dvjDDsAGLK,2018-01-01,Ecuador,1zsG4eaZmkA1dvjDDsAGLK,2017-11-17
1,2,Corazón,"Maluma, Nego do Borel",25743,4lESS6vuruP6a79KWRaQou,2018-01-01,Ecuador,4lESS6vuruP6a79KWRaQou,2017-11-03
2,3,Déjala Que Vuelva (feat. Manuel Turizo),Piso 21,24315,33bnxcjePIkcmNjEFTJX0l,2018-01-01,Ecuador,33bnxcjePIkcmNjEFTJX0l,2017-10-20
3,4,Criminal,"Natti Natasha, Ozuna",22060,6Za3190Sbw39BBC77WSS1C,2018-01-01,Ecuador,6Za3190Sbw39BBC77WSS1C,2017-11-10
4,5,Mayores,"Becky G, Bad Bunny",18587,7JNh1cfm0eXjqFVOzKLyau,2018-01-01,Ecuador,7JNh1cfm0eXjqFVOzKLyau,2017-07-14


In [14]:
ranking_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18750181 entries, 0 to 18199
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   Position      int64         
 1   Track Name    object        
 2   Artist        object        
 3   Streams       int64         
 4   URL           object        
 5   Date          datetime64[ns]
 6   Region        object        
 7   song_id       object        
 8   release_date  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 1.4+ GB


## Get songs' audio features

In [15]:
song_features_df= pd.read_csv(os.path.join('raw_data', 'songs_features_2017_2022.csv'), index_col=0)

In [16]:
song_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.724,0.895,7,-3.617,1,0.0411,0.0322,0.000000,0.0970,0.640,96.022,audio_features,1zsG4eaZmkA1dvjDDsAGLK,spotify:track:1zsG4eaZmkA1dvjDDsAGLK,https://api.spotify.com/v1/tracks/1zsG4eaZmkA1...,https://api.spotify.com/v1/audio-analysis/1zsG...,173720,4
1,0.871,0.768,4,-5.401,0,0.1220,0.2830,0.000023,0.1620,0.765,98.989,audio_features,4lESS6vuruP6a79KWRaQou,spotify:track:4lESS6vuruP6a79KWRaQou,https://api.spotify.com/v1/tracks/4lESS6vuruP6...,https://api.spotify.com/v1/audio-analysis/4lES...,184067,4
2,0.681,0.788,1,-4.323,1,0.0785,0.0482,0.000000,0.0753,0.839,170.019,audio_features,33bnxcjePIkcmNjEFTJX0l,spotify:track:33bnxcjePIkcmNjEFTJX0l,https://api.spotify.com/v1/tracks/33bnxcjePIkc...,https://api.spotify.com/v1/audio-analysis/33bn...,220117,4
3,0.814,0.813,2,-3.023,0,0.0561,0.0300,0.000093,0.2550,0.839,79.997,audio_features,6Za3190Sbw39BBC77WSS1C,spotify:track:6Za3190Sbw39BBC77WSS1C,https://api.spotify.com/v1/tracks/6Za3190Sbw39...,https://api.spotify.com/v1/audio-analysis/6Za3...,232550,4
4,0.743,0.855,5,-3.976,0,0.0436,0.1410,0.004500,0.0990,0.871,96.989,audio_features,7JNh1cfm0eXjqFVOzKLyau,spotify:track:7JNh1cfm0eXjqFVOzKLyau,https://api.spotify.com/v1/tracks/7JNh1cfm0eXj...,https://api.spotify.com/v1/audio-analysis/7JNh...,202453,4


In [17]:
song_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108058 entries, 0 to 2452
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   danceability      108058 non-null  float64
 1   energy            108058 non-null  float64
 2   key               108058 non-null  int64  
 3   loudness          108058 non-null  float64
 4   mode              108058 non-null  int64  
 5   speechiness       108058 non-null  float64
 6   acousticness      108058 non-null  float64
 7   instrumentalness  108058 non-null  float64
 8   liveness          108058 non-null  float64
 9   valence           108058 non-null  float64
 10  tempo             108058 non-null  float64
 11  type              108058 non-null  object 
 12  id                108058 non-null  object 
 13  uri               108058 non-null  object 
 14  track_href        108058 non-null  object 
 15  analysis_url      108058 non-null  object 
 16  duration_ms       1080

In [18]:
processed_songs= song_features_df['id'].unique()

In [19]:
target_songs = ranking_features_df['song_id'].unique()
target_songs= set(target_songs)-set(processed_songs)

In [20]:
print(f'Songs to process: {len(target_songs)}')

Songs to process: 308


In [21]:
song_features_lst = []
for song_id in tqdm(target_songs):
    try:
        song_features = sp.audio_features(song_id)[0]
        if song_features:
            song_features_lst.append(song_features)
    except:
        print("Error with song:", song_id)

  0%|          | 0/308 [00:00<?, ?it/s]

In [22]:
new_song_features_df = pd.DataFrame(song_features_lst)

In [23]:
new_song_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.714,0.531,0,-8.023,1,0.0484,0.0204,0.0,0.1710,0.480,101.003,audio_features,02c4wpeXlXxT7Cekkz3kP3,spotify:track:02c4wpeXlXxT7Cekkz3kP3,https://api.spotify.com/v1/tracks/02c4wpeXlXxT...,https://api.spotify.com/v1/audio-analysis/02c4...,227023,4
1,0.768,0.472,4,-10.047,0,0.0493,0.4750,0.0,0.1100,0.145,110.024,audio_features,68ZVCePM9IP0MSbcmeZuup,spotify:track:68ZVCePM9IP0MSbcmeZuup,https://api.spotify.com/v1/tracks/68ZVCePM9IP0...,https://api.spotify.com/v1/audio-analysis/68ZV...,206167,4
2,0.774,0.571,5,-4.403,0,0.1080,0.2450,0.0,0.1140,0.589,140.030,audio_features,1Xg4v6EF4vXaTQbB9iqOfD,spotify:track:1Xg4v6EF4vXaTQbB9iqOfD,https://api.spotify.com/v1/tracks/1Xg4v6EF4vXa...,https://api.spotify.com/v1/audio-analysis/1Xg4...,181714,4
3,0.540,0.541,6,-10.239,0,0.4520,0.6250,0.0,0.1540,0.294,139.414,audio_features,6sSxF8scYLShxwFCfgRQk1,spotify:track:6sSxF8scYLShxwFCfgRQk1,https://api.spotify.com/v1/tracks/6sSxF8scYLSh...,https://api.spotify.com/v1/audio-analysis/6sSx...,61714,4
4,0.517,0.762,11,-5.806,0,0.0454,0.1610,0.0,0.0897,0.351,156.928,audio_features,0BfVKJALJjpzNYIQiEgF2G,spotify:track:0BfVKJALJjpzNYIQiEgF2G,https://api.spotify.com/v1/tracks/0BfVKJALJjpz...,https://api.spotify.com/v1/audio-analysis/0BfV...,241292,4


In [24]:
new_song_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      291 non-null    float64
 1   energy            291 non-null    float64
 2   key               291 non-null    int64  
 3   loudness          291 non-null    float64
 4   mode              291 non-null    int64  
 5   speechiness       291 non-null    float64
 6   acousticness      291 non-null    float64
 7   instrumentalness  291 non-null    float64
 8   liveness          291 non-null    float64
 9   valence           291 non-null    float64
 10  tempo             291 non-null    float64
 11  type              291 non-null    object 
 12  id                291 non-null    object 
 13  uri               291 non-null    object 
 14  track_href        291 non-null    object 
 15  analysis_url      291 non-null    object 
 16  duration_ms       291 non-null    int64  
 1

Append new songs' features to existing ones

In [25]:
song_features_df = pd.concat([song_features_df, new_song_features_df], axis=0)

In [26]:
song_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.724,0.895,7,-3.617,1,0.0411,0.0322,0.000000,0.0970,0.640,96.022,audio_features,1zsG4eaZmkA1dvjDDsAGLK,spotify:track:1zsG4eaZmkA1dvjDDsAGLK,https://api.spotify.com/v1/tracks/1zsG4eaZmkA1...,https://api.spotify.com/v1/audio-analysis/1zsG...,173720,4
1,0.871,0.768,4,-5.401,0,0.1220,0.2830,0.000023,0.1620,0.765,98.989,audio_features,4lESS6vuruP6a79KWRaQou,spotify:track:4lESS6vuruP6a79KWRaQou,https://api.spotify.com/v1/tracks/4lESS6vuruP6...,https://api.spotify.com/v1/audio-analysis/4lES...,184067,4
2,0.681,0.788,1,-4.323,1,0.0785,0.0482,0.000000,0.0753,0.839,170.019,audio_features,33bnxcjePIkcmNjEFTJX0l,spotify:track:33bnxcjePIkcmNjEFTJX0l,https://api.spotify.com/v1/tracks/33bnxcjePIkc...,https://api.spotify.com/v1/audio-analysis/33bn...,220117,4
3,0.814,0.813,2,-3.023,0,0.0561,0.0300,0.000093,0.2550,0.839,79.997,audio_features,6Za3190Sbw39BBC77WSS1C,spotify:track:6Za3190Sbw39BBC77WSS1C,https://api.spotify.com/v1/tracks/6Za3190Sbw39...,https://api.spotify.com/v1/audio-analysis/6Za3...,232550,4
4,0.743,0.855,5,-3.976,0,0.0436,0.1410,0.004500,0.0990,0.871,96.989,audio_features,7JNh1cfm0eXjqFVOzKLyau,spotify:track:7JNh1cfm0eXjqFVOzKLyau,https://api.spotify.com/v1/tracks/7JNh1cfm0eXj...,https://api.spotify.com/v1/audio-analysis/7JNh...,202453,4


In [27]:
song_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108349 entries, 0 to 290
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   danceability      108349 non-null  float64
 1   energy            108349 non-null  float64
 2   key               108349 non-null  int64  
 3   loudness          108349 non-null  float64
 4   mode              108349 non-null  int64  
 5   speechiness       108349 non-null  float64
 6   acousticness      108349 non-null  float64
 7   instrumentalness  108349 non-null  float64
 8   liveness          108349 non-null  float64
 9   valence           108349 non-null  float64
 10  tempo             108349 non-null  float64
 11  type              108349 non-null  object 
 12  id                108349 non-null  object 
 13  uri               108349 non-null  object 
 14  track_href        108349 non-null  object 
 15  analysis_url      108349 non-null  object 
 16  duration_ms       10834

In [28]:
song_features_df.to_csv(os.path.join('raw_data', 'songs_features_2017_2022.csv'))

## Get songs' genres

In [5]:
songs_genres_df= pd.read_csv(os.path.join('raw_data', 'songs_genres_2017_2021.csv'), index_col=0)
songs_genres_df.columns= 'genre_1 genre_2 genre_3 genre_4 genre_5 genre_6 genre_7 genre_8'.split()
songs_genres_df.tail()

,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8
73kB1ylFaJ2w0HDwxIDeZv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0XJMS3oV6rv69jZElWweA8,r-n-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0QcVKhVp9iHGZ1pj04yTbO,j-pop,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4z2XPrpHz1raJibarWFY7Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78zWaTIgg2GNkgGLA0VK7s,hip-hop,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
processed_songs= songs_genres_df.index

In [7]:
target_songs = ranking_features_df['song_id'].unique()

In [8]:
target_songs= set(target_songs)-set(processed_songs)

In [9]:
all_genres = sp.recommendation_genre_seeds()
all_genres= all_genres['genres']
all_genres += ['rap', 'pop rock', 'trap', 'ranchera', 'corrido', 'cumbia']
all_genres

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [10]:
gender_converter = {'r&b':'r-n-b', 'soundtrack':'soundtracks', 'ost':'soundtracks'}

In [11]:
print("Total songs to be crawled {}".format(len(target_songs)))

Total songs to be crawled 82082


In [12]:
song_genres_dict = {}
counter= 0
for song_id in tqdm(target_songs):
    #try:
    track_info= sp.track(song_id)
    track_genres = []
    for artist in track_info['artists']:
        artist_id=artist['id']
        artist_info= sp.artist(artist_id)
        artist_genres = ['hip-hop' if 'hip hop' in g else g for g in artist_info['genres']]
        artist_genres = [gender_converter.get(g,g) for g in artist_genres]
         
        track_genres += artist_genres #artist_info['genres']
        
        
    filtered_genres = [g for g in  track_genres if g in all_genres]
    
    if len(filtered_genres)==0:
        #print(track_genres)
        split_genres = [g.split(' ')[-1] for g in track_genres if len(g.split(' '))>1]
        split_genres += [g.split(' ')[0] for g in track_genres if len(g.split(' '))>1]
        split_genres = [gender_converter.get(g,g) for g in split_genres]
        
        filtered_genres = [g for g in split_genres if g in all_genres]

    if len(filtered_genres)==0:
        print(f'No genres from {track_genres}')
    song_genres_dict[song_id]= set(filtered_genres)
    sleep(0.01)
    
    if counter>= 300:
        print("Update stored songs' genres. ", end="")
        genres_aux = pd.DataFrame.from_dict(song_genres_dict, orient='index')
        rename_cols= {}
        for i in genres_aux.columns:
            rename_cols[i]= 'genre_{}'.format(i+1)
       
        genres_aux= genres_aux.rename(columns=rename_cols)
        for i in range(len(genres_aux.columns),6):
            genres_aux['genre_{}'.format(i+1)]= np.NAN
        
        songs_genres_df = pd.concat([songs_genres_df, genres_aux], axis=0)
        songs_genres_df.to_csv(os.path.join('raw_data', 'songs_genres_2017_2021.csv'))
        song_genres_dict= {}
        counter=0
        print("DONE!")
    counter += 1

  0%|          | 0/82082 [00:00<?, ?it/s]

No genres from ['opm']
No genres from ['cueca chilena']
No genres from ['grupera', 'gruperas inmortales', 'musica mexicana', 'tropical']
No genres from ['chanson virale', 'chanson virale']
No genres from []
No genres from ['mgpjr', 'talentkonkurranse', 'talentkonkurranse']
No genres from ['muziek voor kinderen']
No genres from []
No genres from ['hoerspiel']
No genres from ['hollywood', 'show tunes', 'broadway', 'hollywood', 'show tunes']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['canto popular uruguayo', 'folklore uruguayo', 'musica popular uruguaya']
No genres from []
No genres from ['opm']
No genres from ['villancicos']
No genres from []
No genres from []
No genres from ['musica costarricense']
No genres from ['hk-pop']
No genres from []
No genres from []
Update stored songs' genres. DONE!
No genres from ['muziek voor kinderen']
No genres from ['danspunk', 'danspunk', 'russelater']
No genres from ['musica mexicana']
No genres from []
No 

No genres from ['koledy']
No genres from ['danish electropop']
No genres from ['drill espanol', 'urbano espanol']
No genres from ['gyerekdalok']
No genres from []
No genres from []
No genres from ['hoerspiel']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['greek drill']
No genres from []
No genres from ['huzunlu sarkilar', 'turkce slow sarkilar']
No genres from ['cueca chilena', 'folklore chileno']
No genres from ['cueca chilena']
Update stored songs' genres. DONE!
No genres from ['drift phonk']
No genres from []
No genres from []
No genres from []
No genres from ['carnaval']
No genres from ['hollywood']
No genres from []
No genres from []
No genres from []
No genres from ['eurovision']
No genres from ['banda', 'musica mexicana', 'norteno']
No genres from ['opm']
No genres from []
No genres from ['christmas instrumental']
No genres from ['barnmusik', 'barnmusik']
No genres from ['urbano espanol']
No genres from []
No genres from ['cueca chilena

No genres from []
No genres from ['aussie drill']
No genres from []
No genres from []
No genres from []
No genres from ['greek drill']
No genres from []
No genres from []
Update stored songs' genres. DONE!
No genres from ['musica costarricense']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['bornesange']
No genres from []
No genres from ['barnmusik']
No genres from []
No genres from []
No genres from []
No genres from []
No genres from ['chinese new year']
No genres from ['mgp']
No genres from []
No genres from []
No genres from []
No genres from ['tecnobrega']
No genres from ['musica etnica']
No genres from []
No genres from []
No genres from []
No genres from ['hollywood']
No genres from []
No genres from ['danspunk', 'russelater']
No genres from []
No genres from []
No genres from []
No genres from ['koledy']
No genres from []
No genres from []
No genres from ['fotbollslatar']
No genres from []
No genres from ['muziek voor kinderen', 'christ

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
genres_aux = pd.DataFrame.from_dict(song_genres_dict, orient='index')
songs_genres_df = pd.concat(songs_genres_df, genres_aux, axis=0)
song_genres_df.to_csv(os.path.join('generated_data', 'songs_genres_2017_2022.csv'))        

In [ ]:
print("That's all folks!")